In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from imblearn import over_sampling
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
plt.rcParams['font.sans-serif']='SimHei'
plt.rcParams['axes.unicode_minus']=False

In [ ]:
#导入数据
def load_data(path):
    predata=pd.read_csv(path)
    return predata

In [ ]:
train_data=load_data(r'/kaggle/input/health-insurance-cross-sell-prediction/train.csv')
train_data

In [ ]:
#检查缺失值
train_data.info()

In [ ]:
#重复值
train_data.duplicated().sum()

In [ ]:
#数据分布
def plot_frequency(predata,num):
    columns=list(predata.columns)

    fig=plt.figure(figsize=(16,16))
    for i in range(num):
        if i==0:
            plt.subplot(4,4,i+1)
            continue
        else:
            plt.subplot(4,4,i+1)
            plt.hist(predata.iloc[:,i])
            plt.xlabel(columns[i])
            plt.ylabel('Frequency')
            plt.title(columns[i]+' Frequency')
            plt.tight_layout()    
    plt.show()

In [ ]:
plot_frequency(train_data,12)
#样本不均衡

# 特征：
0    id
1 定性 Gender 性别                  female\male
2 定量 Age 年龄
3 定性 Drving_License 是否有驾照          0\1
4 定量 Region_Code 地区代码
5 定性 Previously_Insured 先前是否有保险     0\1
6 有序 Vehicle_Age 车龄               >2year\1-2year\<1year
7 定性 Vehicle_Damage 车辆损坏           yes\no
8 定量 Annual_Premium 年保费
9 定量 Policy_Sales_Channel 采购渠道
10定量 Vintage 客户与公司关联的天数
11定性 Response 是否感兴趣              0\1

In [ ]:
#对定性数据编码
def code_DX(predata):
    ohe=preprocessing.OneHotEncoder()
    ohedata=pd.concat([predata.iloc[:,1],predata.iloc[:,7]],axis=1)
    ohe=ohe.fit(ohedata)
    result1=ohe.transform(ohedata).toarray() 
    ohecols=['Gender_Female', 'Gender_Male', 'Vehicle_Damage_No', 'Vehicle_Damage_Yes']
    return result1,ohecols
result1,ohecols=code_DX(train_data)
print(result1)
print(ohecols)

In [ ]:
#对有序数据编码
def code_YX(predata):
    x=np.array(predata.iloc[:,6])
    for i in range(len(x)):
        if x[i]=='< 1 Year':
            x[i]=1
        if x[i]=='1-2 Year':
            x[i]=2
        if x[i]=='> 2 Years':
            x[i]=3
    xcol=['Vehicle_Age']
    return x,xcol
x,xcol=code_YX(train_data)
print(x)
print(xcol)

In [ ]:
#对定量数据编码
#将年龄转为有序变量
def code_age_to_YX(predata):
    age=np.array(predata.iloc[:,2])
    for i in range(len(age)):
        if age[i]>=20&age[i]<30:
            age[i]=1
        elif age[i]>=30&age[i]<40:
            age[i]=2
        elif age[i]>=40&age[i]<50:
            age[i]=3
        elif age[i]>=50&age[i]<60:
            age[i]=4
        elif age[i]>=70&age[i]<80:
            age[i]=5
        else:
            age[i]=6
    agecol=['Age']
    return age,agecol
age,agecol=code_age_to_YX(train_data)
print(age)
print(agecol)

In [ ]:
#定量数据标准化
def standard(predata):
    scaler=preprocessing.StandardScaler()
    result3=scaler.fit_transform(pd.concat([predata.loc[:,'Annual_Premium'],predata.loc[:,'Vintage'],predata.loc[:,'Policy_Sales_Channel']],axis=1))
    scalercol=['Annual_Premium','Vintage','Policy_Sales_Channel']
    return result3,scalercol
result3,scalercol=standard(train_data)
print(result3)
print(scalercol)

In [ ]:
#将地区代号转为定性变量
def code_region_to_DX(predata):
    code=np.array(predata.iloc[:,4])
    for i in range(len(code)):
        if (code[i]>=1)&(code[i]<10):
            code[i]=1
        elif (code[i]>=10)&(code[i]<20):
            code[i]=2
        elif (code[i]>=20)&(code[i]<30):
            code[i]=3
        elif (code[i]>=30)&(code[i]<40):
            code[i]=4
        else:
            code[i]=6
    ohe=preprocessing.OneHotEncoder()
    ohe=ohe.fit(code.reshape(-1,1))
    result4=ohe.transform(code.reshape(-1,1)).toarray() 
    regincols=['Region_Code_10', 'Region_Code_20', 'Region_Code_30', 'Region_Code_40','Region_Code_max']
    return result4,regincols
result4,regincols=code_region_to_DX(train_data)
print(result4)
print(regincols)

In [ ]:
#不用处理的
def no_pro(predata,state):
    noprocess=np.array(pd.concat([predata.iloc[:,3],predata.iloc[:,5]],axis=1))    
    id=np.array(predata.iloc[:,0])
    noprocesscol=['Driving_License','Previously_Insured']
    if state==1:
        ycol=['Response']
        y=np.array(predata.iloc[:,-1])
        return noprocess,y,noprocesscol,ycol
    else:
        return noprocess,noprocesscol
noprocess,y,noprocesscol,ycol=no_pro(train_data,1)

In [ ]:
#转为单个表
def trans_add(result1,result3,age,noprocess,result4,x,y,ohecols,scalercol,agecol,noprocesscol,regincols,xcol,ycol,state):
    df1=pd.DataFrame(data=result1,columns=ohecols)
    df3=pd.DataFrame(data=result3,columns=scalercol)
    df_age=pd.DataFrame(data=age,columns=agecol)
    df_no=pd.DataFrame(data=noprocess,columns=noprocesscol)
    df4=pd.DataFrame(data=result4,columns=regincols)
    df_x=pd.DataFrame(data=x,columns=xcol)
    if state==1:
        df_y=pd.DataFrame(data=y,columns=ycol)   
        data=pd.concat([df_age,df1,df_no,df4,df_x,df3,df_y],axis=1)
    else:
        data=pd.concat([df_age,df1,df_no,df4,df_x,df3],axis=1)
    return data
data=trans_add(result1,result3,age,noprocess,result4,x,y,ohecols,scalercol,agecol,noprocesscol,regincols,xcol,ycol,1)

In [ ]:
#划分数据集
def split(data):
    
    x=data.iloc[:,:-1]
    y=data.iloc[:,-1]
    return x,y

x,y=split(data)

In [ ]:
#样本均衡化
ros = over_sampling.RandomOverSampler(random_state=0)
x_resampled, y_resampled = ros.fit_resample(x, y)
print(y_resampled.value_counts())

**构建模型**

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x_resampled,y_resampled,test_size=0.3)
print(xtrain.shape,ytrain.shape)
print(xtest.shape,ytest.shape)

In [ ]:
rfc=RandomForestClassifier(random_state=100,n_estimators=50)
rfc=rfc.fit(xtrain,ytrain)
rfc_s=cross_val_score(rfc,xtrain,ytrain,cv=10,scoring='roc_auc')
rfc_s.mean()

In [ ]:
ypred_rfc=rfc.predict(xtest)
rac_rcf=roc_auc_score(ytest,ypred_rfc)
rac_rcf

In [ ]:
from sklearn.metrics import recall_score,precision_score,classification_report
rs=recall_score(ytest,ypred_rfc)
ps=precision_score(ytest,ypred_rfc)
print('召回率：',rs)
print('精确率：',ps)
print(classification_report(ytest,ypred_rfc))

**对test.csv处理**

In [ ]:
test_data=load_data(r'/kaggle/input/health-insurance-cross-sell-prediction/test.csv')
test_data

In [ ]:
test_data.info()

In [ ]:
plot_frequency(test_data,11)

In [ ]:
result1_,ohecols_=code_DX(test_data)
x_,xcol_=code_YX(test_data)
age_,agecol_=code_age_to_YX(test_data)
result3_,scalercol_=standard(test_data)
result4_,regincols_=code_region_to_DX(test_data)
noprocess_,noprocesscol_=no_pro(test_data,0)
data_=trans_add(result1_,result3_,age_,noprocess_,result4_,x_,y,ohecols_,scalercol_,agecol_,noprocesscol_,regincols_,xcol_,ycol,0)

In [ ]:
data_

In [ ]:
test_pred=rfc.predict(data_)
test_pred

In [ ]:
data_['Response']=test_pred
data_

In [ ]:
result_pred=pd.concat([test_data.iloc[:,0],data_.iloc[:,-1]],axis=1)
result_pred

In [ ]:
result_pred.to_csv(r'/kaggle/working/result.csv',index=False)